In [1]:
import sys
sys.path.append('../../../')
from utils.packages import *
from utils.ml_fairness import *
from utils.standard_data import *
dir = 'res/adult4/'
Path(dir).mkdir(parents=True, exist_ok=True)

d_fields = ['Stage', 'CVR', 'CVD', 'V_SPD', 'V_EOD', 'V_AOD', 'V_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
diff_file = dir + 'diff' + '.csv'
if(not os.path.isfile(diff_file)):
    with open(diff_file, 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(d_fields)
    
f_count = len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name)) and not name.startswith('.')])
# fields = ['Acc', 'F1', 'DI','SPD', 'EOD', 'AOD', 'ERD', 'CNT', 'TI']
fields = ['Acc', 'F1', 'SPD', 'EOD', 'AOD', 'ERD']

filename = dir + 'glob-' + str(f_count) + '.csv'
with open(filename, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)


In [2]:
train_path = '../../../data/adult/adult.data'
test_path = '../../../data/adult/adult.test'

column_names = ['age', 'workclass', 'fnlwgt', 'education',
            'education-num', 'marital-status', 'occupation', 'relationship',
            'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
            'native-country', 'income-per-year']
na_values=['?']

train = pd.read_csv(train_path, header=None, names=column_names, 
                    skipinitialspace=True, na_values=na_values)
test = pd.read_csv(test_path, header=0, names=column_names,
                   skipinitialspace=True, na_values=na_values)

df = pd.concat([test, train], ignore_index=True)

##### Drop na values
dropped = df.dropna()
count = df.shape[0] - dropped.shape[0]
print("Missing Data: {} rows removed.".format(count))
df = dropped

# Create a one-hot encoding of the categorical variables.
cat_feat = ['sex', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'native-country']
# df = pd.get_dummies(df, columns=cat_feat, prefix_sep='=')

## Implement label encoder instead of one-hot encoder
for feature in cat_feat:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])

Missing Data: 3620 rows removed.


In [3]:
seed = randrange(100)
y2_train, y2_test = train_test_split(df, test_size = 0.3, random_state = seed) # stratify=df['race']
y1_train, y1_test = train_test_split(df, test_size = 0.3, random_state = seed) # 

pro_att_name = ['race'] # ['race', 'sex']
priv_class = ['White'] # ['White', 'Male']
reamining_cat_feat = []

y2_data_orig_train, y2_X_train, y2_y_train = load_adult_data(y2_train, pro_att_name, priv_class, reamining_cat_feat)
y2_data_orig_test, y2_X_test, y2_y_test = load_adult_data(y2_test, pro_att_name, priv_class, reamining_cat_feat)

y1_data_orig_train, y1_X_train, y1_y_train = load_adult_data(y1_train, pro_att_name, priv_class, reamining_cat_feat)
y1_data_orig_test, y1_X_test, y1_y_test = load_adult_data(y1_test, pro_att_name, priv_class, reamining_cat_feat)


In [4]:
sc = StandardScaler()

trained = sc.fit(y2_X_train)
y2_X_train = trained.transform(y2_X_train)
y2_X_test = trained.transform(y2_X_test)

y2_data_orig_train.features = y2_X_train
y2_data_orig_test.features = y2_X_test

y1_X_train = trained.transform(y1_X_train)
y1_X_test = trained.transform(y1_X_test)

y1_data_orig_train.features = y1_X_train
y1_data_orig_test.features = y1_X_test

In [5]:
pca = PCA(n_components=14)

trained = pca.fit(y2_X_train)
y2_X_train = trained.transform(y2_X_train)
y2_X_test = trained.transform(y2_X_test)
y2_data_orig_test.features = y2_X_test

y1_X_train = trained.transform(y1_X_train)
y1_X_test = trained.transform(y1_X_test)
y1_data_orig_test.features = y1_X_test

In [6]:
from catboost import CatBoostClassifier
y2_model = CatBoostClassifier()
y2_mdl = y2_model.fit(y2_X_train, y2_y_train)

y1_model = CatBoostClassifier()
y1_mdl = y1_model.fit(y1_X_train, y1_y_train)


Learning rate set to 0.045043
0:	learn: 0.6608721	total: 194ms	remaining: 3m 13s
1:	learn: 0.6360562	total: 325ms	remaining: 2m 41s
2:	learn: 0.6125359	total: 354ms	remaining: 1m 57s
3:	learn: 0.5911210	total: 431ms	remaining: 1m 47s
4:	learn: 0.5727370	total: 459ms	remaining: 1m 31s
5:	learn: 0.5552029	total: 487ms	remaining: 1m 20s
6:	learn: 0.5401901	total: 516ms	remaining: 1m 13s
7:	learn: 0.5242095	total: 542ms	remaining: 1m 7s
8:	learn: 0.5087245	total: 570ms	remaining: 1m 2s
9:	learn: 0.4957945	total: 595ms	remaining: 58.9s
10:	learn: 0.4867218	total: 624ms	remaining: 56.1s
11:	learn: 0.4761847	total: 647ms	remaining: 53.3s
12:	learn: 0.4685477	total: 671ms	remaining: 51s
13:	learn: 0.4600707	total: 697ms	remaining: 49.1s
14:	learn: 0.4523689	total: 721ms	remaining: 47.4s
15:	learn: 0.4463651	total: 754ms	remaining: 46.4s
16:	learn: 0.4413224	total: 778ms	remaining: 45s
17:	learn: 0.4366725	total: 801ms	remaining: 43.7s
18:	learn: 0.4328227	total: 835ms	remaining: 43.1s
19:	lear

164:	learn: 0.3285437	total: 5.71s	remaining: 28.9s
165:	learn: 0.3282992	total: 5.75s	remaining: 28.9s
166:	learn: 0.3281454	total: 5.77s	remaining: 28.8s
167:	learn: 0.3279302	total: 5.8s	remaining: 28.7s
168:	learn: 0.3278363	total: 5.84s	remaining: 28.7s
169:	learn: 0.3277329	total: 5.87s	remaining: 28.6s
170:	learn: 0.3275823	total: 5.9s	remaining: 28.6s
171:	learn: 0.3274556	total: 5.92s	remaining: 28.5s
172:	learn: 0.3272192	total: 5.95s	remaining: 28.4s
173:	learn: 0.3271109	total: 5.97s	remaining: 28.4s
174:	learn: 0.3268114	total: 6s	remaining: 28.3s
175:	learn: 0.3266526	total: 6.03s	remaining: 28.2s
176:	learn: 0.3264975	total: 6.06s	remaining: 28.2s
177:	learn: 0.3262579	total: 6.08s	remaining: 28.1s
178:	learn: 0.3260923	total: 6.11s	remaining: 28s
179:	learn: 0.3259438	total: 6.14s	remaining: 28s
180:	learn: 0.3258265	total: 6.17s	remaining: 27.9s
181:	learn: 0.3257303	total: 6.21s	remaining: 27.9s
182:	learn: 0.3256363	total: 6.24s	remaining: 27.9s
183:	learn: 0.3254867

325:	learn: 0.3067383	total: 12.4s	remaining: 25.6s
326:	learn: 0.3066687	total: 12.4s	remaining: 25.6s
327:	learn: 0.3065449	total: 12.5s	remaining: 25.5s
328:	learn: 0.3064356	total: 12.5s	remaining: 25.5s
329:	learn: 0.3063267	total: 12.5s	remaining: 25.4s
330:	learn: 0.3062221	total: 12.5s	remaining: 25.4s
331:	learn: 0.3061012	total: 12.6s	remaining: 25.3s
332:	learn: 0.3059669	total: 12.6s	remaining: 25.3s
333:	learn: 0.3058623	total: 12.6s	remaining: 25.2s
334:	learn: 0.3057552	total: 12.7s	remaining: 25.2s
335:	learn: 0.3056352	total: 12.7s	remaining: 25.1s
336:	learn: 0.3054978	total: 12.7s	remaining: 25.1s
337:	learn: 0.3053567	total: 12.8s	remaining: 25s
338:	learn: 0.3052361	total: 12.8s	remaining: 25s
339:	learn: 0.3051552	total: 12.9s	remaining: 25s
340:	learn: 0.3050203	total: 12.9s	remaining: 24.9s
341:	learn: 0.3048919	total: 12.9s	remaining: 24.9s
342:	learn: 0.3048081	total: 13s	remaining: 24.8s
343:	learn: 0.3046633	total: 13s	remaining: 24.8s
344:	learn: 0.3045523	

491:	learn: 0.2897658	total: 17.6s	remaining: 18.2s
492:	learn: 0.2896700	total: 17.6s	remaining: 18.1s
493:	learn: 0.2895913	total: 17.7s	remaining: 18.1s
494:	learn: 0.2894542	total: 17.7s	remaining: 18s
495:	learn: 0.2893691	total: 17.7s	remaining: 18s
496:	learn: 0.2893013	total: 17.7s	remaining: 17.9s
497:	learn: 0.2892293	total: 17.8s	remaining: 17.9s
498:	learn: 0.2891476	total: 17.8s	remaining: 17.9s
499:	learn: 0.2890610	total: 17.8s	remaining: 17.8s
500:	learn: 0.2889816	total: 17.8s	remaining: 17.8s
501:	learn: 0.2889010	total: 17.9s	remaining: 17.7s
502:	learn: 0.2888068	total: 17.9s	remaining: 17.7s
503:	learn: 0.2887335	total: 17.9s	remaining: 17.6s
504:	learn: 0.2886715	total: 17.9s	remaining: 17.6s
505:	learn: 0.2886156	total: 18s	remaining: 17.5s
506:	learn: 0.2885283	total: 18s	remaining: 17.5s
507:	learn: 0.2884131	total: 18s	remaining: 17.4s
508:	learn: 0.2883255	total: 18s	remaining: 17.4s
509:	learn: 0.2882357	total: 18.1s	remaining: 17.4s
510:	learn: 0.2881864	to

653:	learn: 0.2762919	total: 21.8s	remaining: 11.5s
654:	learn: 0.2762066	total: 21.8s	remaining: 11.5s
655:	learn: 0.2761447	total: 21.9s	remaining: 11.5s
656:	learn: 0.2760839	total: 21.9s	remaining: 11.4s
657:	learn: 0.2759949	total: 21.9s	remaining: 11.4s
658:	learn: 0.2759225	total: 21.9s	remaining: 11.4s
659:	learn: 0.2758382	total: 22s	remaining: 11.3s
660:	learn: 0.2757536	total: 22s	remaining: 11.3s
661:	learn: 0.2756851	total: 22s	remaining: 11.2s
662:	learn: 0.2756175	total: 22s	remaining: 11.2s
663:	learn: 0.2755528	total: 22.1s	remaining: 11.2s
664:	learn: 0.2754739	total: 22.1s	remaining: 11.1s
665:	learn: 0.2754065	total: 22.1s	remaining: 11.1s
666:	learn: 0.2753433	total: 22.1s	remaining: 11.1s
667:	learn: 0.2752558	total: 22.2s	remaining: 11s
668:	learn: 0.2752059	total: 22.2s	remaining: 11s
669:	learn: 0.2751158	total: 22.2s	remaining: 10.9s
670:	learn: 0.2750480	total: 22.2s	remaining: 10.9s
671:	learn: 0.2749736	total: 22.3s	remaining: 10.9s
672:	learn: 0.2749003	to

...

297:	learn: 0.3100682	total: 5.81s	remaining: 13.7s
298:	learn: 0.3099751	total: 5.83s	remaining: 13.7s
299:	learn: 0.3098201	total: 5.84s	remaining: 13.6s
300:	learn: 0.3097163	total: 5.86s	remaining: 13.6s
301:	learn: 0.3095786	total: 5.88s	remaining: 13.6s
302:	learn: 0.3094713	total: 5.89s	remaining: 13.6s
303:	learn: 0.3092914	total: 5.91s	remaining: 13.5s
304:	learn: 0.3091667	total: 5.93s	remaining: 13.5s
305:	learn: 0.3090549	total: 5.95s	remaining: 13.5s
306:	learn: 0.3089476	total: 5.97s	remaining: 13.5s
307:	learn: 0.3088553	total: 5.99s	remaining: 13.4s
308:	learn: 0.3087437	total: 6s	remaining: 13.4s
309:	learn: 0.3085870	total: 6.02s	remaining: 13.4s
310:	learn: 0.3084409	total: 6.04s	remaining: 13.4s
311:	learn: 0.3083261	total: 6.06s	remaining: 13.4s
312:	learn: 0.3082386	total: 6.08s	remaining: 13.3s
313:	learn: 0.3080649	total: 6.09s	remaining: 13.3s
314:	learn: 0.3079650	total: 6.11s	remaining: 13.3s
315:	learn: 0.3078364	total: 6.13s	remaining: 13.3s
316:	learn: 0.3

461:	learn: 0.2925870	total: 8.53s	remaining: 9.93s
462:	learn: 0.2925058	total: 8.54s	remaining: 9.91s
463:	learn: 0.2923759	total: 8.56s	remaining: 9.89s
464:	learn: 0.2922774	total: 8.57s	remaining: 9.87s
465:	learn: 0.2921799	total: 8.59s	remaining: 9.84s
466:	learn: 0.2921179	total: 8.61s	remaining: 9.82s
467:	learn: 0.2920412	total: 8.62s	remaining: 9.8s
468:	learn: 0.2919528	total: 8.64s	remaining: 9.78s
469:	learn: 0.2918248	total: 8.65s	remaining: 9.76s
470:	learn: 0.2917261	total: 8.67s	remaining: 9.74s
471:	learn: 0.2916645	total: 8.68s	remaining: 9.71s
472:	learn: 0.2915747	total: 8.7s	remaining: 9.69s
473:	learn: 0.2914574	total: 8.71s	remaining: 9.67s
474:	learn: 0.2913500	total: 8.73s	remaining: 9.65s
475:	learn: 0.2912533	total: 8.75s	remaining: 9.63s
476:	learn: 0.2911592	total: 8.76s	remaining: 9.61s
477:	learn: 0.2910199	total: 8.78s	remaining: 9.59s
478:	learn: 0.2909573	total: 8.79s	remaining: 9.57s
479:	learn: 0.2908462	total: 8.81s	remaining: 9.55s
480:	learn: 0.

625:	learn: 0.2784611	total: 11s	remaining: 6.54s
626:	learn: 0.2784015	total: 11s	remaining: 6.52s
627:	learn: 0.2783093	total: 11s	remaining: 6.5s
628:	learn: 0.2782350	total: 11s	remaining: 6.49s
629:	learn: 0.2781443	total: 11s	remaining: 6.47s
630:	learn: 0.2780545	total: 11s	remaining: 6.45s
631:	learn: 0.2779689	total: 11s	remaining: 6.42s
632:	learn: 0.2779287	total: 11s	remaining: 6.41s
633:	learn: 0.2778346	total: 11.1s	remaining: 6.39s
634:	learn: 0.2777632	total: 11.1s	remaining: 6.37s
635:	learn: 0.2776859	total: 11.1s	remaining: 6.35s
636:	learn: 0.2776267	total: 11.1s	remaining: 6.33s
637:	learn: 0.2775495	total: 11.1s	remaining: 6.31s
638:	learn: 0.2774955	total: 11.1s	remaining: 6.29s
639:	learn: 0.2773938	total: 11.1s	remaining: 6.27s
640:	learn: 0.2773205	total: 11.2s	remaining: 6.25s
641:	learn: 0.2772579	total: 11.2s	remaining: 6.23s
642:	learn: 0.2771755	total: 11.2s	remaining: 6.21s
643:	learn: 0.2770656	total: 11.2s	remaining: 6.19s
644:	learn: 0.2769649	total: 

787:	learn: 0.2667312	total: 13.2s	remaining: 3.55s
788:	learn: 0.2666692	total: 13.2s	remaining: 3.53s
789:	learn: 0.2665974	total: 13.2s	remaining: 3.51s
790:	learn: 0.2665154	total: 13.2s	remaining: 3.5s
791:	learn: 0.2664347	total: 13.2s	remaining: 3.48s
792:	learn: 0.2663481	total: 13.3s	remaining: 3.46s
793:	learn: 0.2662975	total: 13.3s	remaining: 3.44s
794:	learn: 0.2662410	total: 13.3s	remaining: 3.42s
795:	learn: 0.2662022	total: 13.3s	remaining: 3.41s
796:	learn: 0.2661407	total: 13.3s	remaining: 3.39s
797:	learn: 0.2660772	total: 13.3s	remaining: 3.37s
798:	learn: 0.2660217	total: 13.3s	remaining: 3.35s
799:	learn: 0.2659388	total: 13.3s	remaining: 3.33s
800:	learn: 0.2658676	total: 13.4s	remaining: 3.32s
801:	learn: 0.2658243	total: 13.4s	remaining: 3.3s
802:	learn: 0.2657766	total: 13.4s	remaining: 3.28s
803:	learn: 0.2657047	total: 13.4s	remaining: 3.27s
804:	learn: 0.2656258	total: 13.4s	remaining: 3.25s
805:	learn: 0.2655457	total: 13.4s	remaining: 3.23s
806:	learn: 0.

954:	learn: 0.2557329	total: 15.4s	remaining: 726ms
955:	learn: 0.2556672	total: 15.4s	remaining: 710ms
956:	learn: 0.2555912	total: 15.4s	remaining: 693ms
957:	learn: 0.2555341	total: 15.4s	remaining: 677ms
958:	learn: 0.2554767	total: 15.5s	remaining: 661ms
959:	learn: 0.2554238	total: 15.5s	remaining: 645ms
960:	learn: 0.2553604	total: 15.5s	remaining: 628ms
961:	learn: 0.2552983	total: 15.5s	remaining: 612ms
962:	learn: 0.2552277	total: 15.5s	remaining: 597ms
963:	learn: 0.2551771	total: 15.6s	remaining: 581ms
964:	learn: 0.2551269	total: 15.6s	remaining: 565ms
965:	learn: 0.2550795	total: 15.6s	remaining: 549ms
966:	learn: 0.2549814	total: 15.6s	remaining: 532ms
967:	learn: 0.2549283	total: 15.6s	remaining: 516ms
968:	learn: 0.2548726	total: 15.6s	remaining: 500ms
969:	learn: 0.2548075	total: 15.6s	remaining: 484ms
970:	learn: 0.2547706	total: 15.6s	remaining: 467ms
971:	learn: 0.2546890	total: 15.7s	remaining: 451ms
972:	learn: 0.2546270	total: 15.7s	remaining: 435ms
973:	learn: 

In [7]:
# plot_model_performance(y2_mdl, y2_X_test, y2_y_test)
y1_pred, y1_fair = get_fair_metrics_and_plot(filename, y1_data_orig_test, y1_mdl)
y2_pred, y2_fair = get_fair_metrics_and_plot(filename, y2_data_orig_test, y2_mdl)


y1_fair = y1_fair.drop(['DI', 'CNT', 'TI'], axis=1)
y2_fair = y2_fair.drop(['DI', 'CNT', 'TI'], axis=1)
CVR, CVD, AVR_EOD, AVD_EOD, AVR_SPD, AVD_SPD, AVD_AOD, AV_ERD = compute_new_metrics(y1_data_orig_test, y1_pred, y2_pred)
row_y1 = y1_fair.iloc[[0]].values[0].tolist()
row_y2 = y2_fair.iloc[[0]].values[0].tolist()
diff = []

with open(filename, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_y1)
    csvwriter.writerow(row_y2) 

diff.append(CVR)
diff.append(CVD)
diff.append(AVD_SPD)
diff.append(AVD_EOD)
diff.append(AVD_AOD)
diff.append(AV_ERD)

for i in range(len(row_y2)):
    if(i < 2):
        change = row_y2[i] - row_y1[i]
    else:
        sign = ''
        if(row_y2[i] >= 0 and row_y1[i] >= 0):
            sign = '(+)'
            d = abs(row_y2[i]) - abs(row_y1[i])
        if(row_y2[i] < 0 and row_y1[i] < 0):
            sign = '(-)'
            d = abs(row_y2[i]) - abs(row_y1[i])
        if(row_y2[i] < 0 and row_y1[i] >= 0):
            sign = '(+-)'
            d = row_y2[i] - row_y1[i]
        if(row_y2[i] >=0 and row_y1[i] < 0):
            sign = '(-+)'
            d = row_y2[i] - row_y1[i]
        d = round(d, 3)
        change = sign + ' ' + str(d)

    diff.append(change)

cols = ['CVR', 'CVD ', 'AV_SPD', 'AV_EOD', 'AV_AOD', 'AV_ERD', 'Acc', 'F1','SPD', 'EOD', 'AOD', 'ERD']
# metrics = pd.DataFrame(data=obj_fairness, index=['y1'], columns=cols)
diff_df = pd.DataFrame(data=[diff], columns  = cols, index = ['Diff']).round(3)
stage = 'StandardScaler'
model_name = 'adult4'
diff = diff_df.iloc[0].values.tolist()
diff.insert(0, stage)
diff.insert(0, model_name)
with open(diff_file, 'a') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(diff)    
    diff_df

Unprinv: race 0.0
